# This notebook aims to create a CNN model to train a small dataset of traffic signs

In [49]:
# Import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Activation, Dropout
from keras.layers.convolutional import Conv2D
from keras.optimizers import adam
from keras.layers.pooling import MaxPooling2D

In [9]:
train_folder = "./Datasets/small_train_traffic.p"
test_folder = "./Datasets/small_test_traffic.p"
X_train = pd.read_pickle(train_folder)['features']
Y_train = pd.read_pickle(train_folder)['labels']
X_test = pd.read_pickle(test_folder)['features']
Y_test = pd.read_pickle(test_folder)['labels']
print("**** shape of current data ****")
print(f"Training features: {X_train.shape}")
print(f"Training labels: {Y_train.shape}")
print(f"testing features: {X_test.shape}")
print(f"testing labels: {Y_test.shape}")

**** shape of current data ****
Training features: (100, 32, 32, 3)
Training labels: (100,)
testing features: (20, 32, 32, 3)
testing labels: (20,)


In [16]:
# Normalize features
X_train_norm = (X_train -127.5)/255
X_test_norm = (X_test -127.5)/255
## One hot encode labels
oneHot = LabelBinarizer()
# get parameters of one hot encoding based on the classes
oneHot.fit(Y_train)
# transform data using the one hot encoding parameters
Y_train_oneHot = oneHot.transform(Y_train)
Y_test_oneHot = oneHot.transform(Y_test)
print("****** new dataset shape ******")
print(f"training dataset: {X_train_norm.shape}")
print(f"testing dataset: {X_test_norm.shape}")
print(f"training labels: {Y_train_oneHot.shape}")
print(f"testing labels: {Y_test_oneHot.shape}")

****** new dataset shape ******
training dataset: (100, 32, 32, 3)
testing dataset: (20, 32, 32, 3)
training labels: (100, 5)
testing labels: (20, 5)


# Creating the model architecture

In [33]:
my_model = Sequential()
my_model.add(Conv2D(12, 3, input_shape=(32,32,3)))
my_model.add(Activation("relu"))
my_model.add(Conv2D(6, 5))
my_model.add(Activation("relu"))
my_model.add(Flatten())
my_model.add(Activation("relu"))
my_model.add(Dense(60))
my_model.add(Activation("relu"))
my_model.add(Dense(5))
my_model.add(Activation("softmax"))

In [34]:
my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 30, 30, 12)        336       
_________________________________________________________________
activation_11 (Activation)   (None, 30, 30, 12)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 26, 26, 6)         1806      
_________________________________________________________________
activation_12 (Activation)   (None, 26, 26, 6)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 4056)              0         
_________________________________________________________________
activation_13 (Activation)   (None, 4056)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 60)                243420    
__________

In [35]:
# Set optimizer
my_model.compile(adam(lr=0.0001), "categorical_crossentropy", metrics=['accuracy'])

In [41]:
history = my_model.fit(X_train_norm, Y_train_oneHot, validation_split=0.2)

Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - 0s 1ms/step - loss: 1.0559 - acc: 0.8375 - val_loss: 1.1423 - val_acc: 0.8000
Epoch 2/10
80/80 [==============================] - 0s 1ms/step - loss: 1.0386 - acc: 0.6875 - val_loss: 1.1316 - val_acc: 0.7000
Epoch 3/10
80/80 [==============================] - 0s 1ms/step - loss: 1.0138 - acc: 0.6375 - val_loss: 1.0898 - val_acc: 0.8000
Epoch 4/10
80/80 [==============================] - 0s 1ms/step - loss: 0.9629 - acc: 0.8000 - val_loss: 1.0618 - val_acc: 0.8000
Epoch 5/10
80/80 [==============================] - 0s 1ms/step - loss: 0.9429 - acc: 0.8500 - val_loss: 1.0274 - val_acc: 0.8000
Epoch 6/10
80/80 [==============================] - 0s 1ms/step - loss: 0.9061 - acc: 0.8375 - val_loss: 0.9736 - val_acc: 0.8000
Epoch 7/10
80/80 [==============================] - 0s 1ms/step - loss: 0.8647 - acc: 0.8375 - val_loss: 0.9334 - val_acc: 0.8000
Epoch 8/10
80/80 [============================

In [42]:
# test data
result=my_model.evaluate(X_test_norm, Y_test_oneHot)

20/20 [==============================] - 0s 500us/step


In [48]:
# `results` holds the results for the model metrics parameters set at the beginning
print(my_model.metrics_names)
print(result)
print(f"Test results- {my_model.metrics_names[0]}: {result[0]}, {my_model.metrics_names[1]}: {result[1]}")

['loss', 'acc']
[0.8452666997909546, 0.8500000238418579]
Test results- loss: 0.8452666997909546, acc: 0.8500000238418579


# lets try a different architecture including some regularization (dropout) and pooling layers

In [50]:
# Build Convolutional Pooling Neural Network with Dropout in Keras Here
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(5))
model.add(Activation('softmax'))

In [51]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
activation_16 (Activation)   (None, 15, 15, 32)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 7200)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               921728    
_________________________________________________________________
activation_17 (Activation)   (None, 128)               0         
__________

In [52]:
model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_train_norm, Y_train_oneHot, epochs=10, validation_split=0.2)

Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - 1s 11ms/step - loss: 1.2297 - acc: 0.3875 - val_loss: 0.7367 - val_acc: 0.7000
Epoch 2/10
80/80 [==============================] - 0s 1ms/step - loss: 0.7532 - acc: 0.5750 - val_loss: 0.4464 - val_acc: 1.0000
Epoch 3/10
80/80 [==============================] - 0s 1ms/step - loss: 0.5082 - acc: 0.8250 - val_loss: 0.3086 - val_acc: 1.0000
Epoch 4/10
80/80 [==============================] - 0s 1ms/step - loss: 0.3831 - acc: 0.8375 - val_loss: 0.2251 - val_acc: 0.9500
Epoch 5/10
80/80 [==============================] - 0s 1ms/step - loss: 0.3118 - acc: 0.8875 - val_loss: 0.1764 - val_acc: 1.0000
Epoch 6/10
80/80 [==============================] - 0s 1ms/step - loss: 0.2453 - acc: 0.9250 - val_loss: 0.1488 - val_acc: 1.0000
Epoch 7/10
80/80 [==============================] - 0s 1ms/step - loss: 0.2217 - acc: 0.9750 - val_loss: 0.1535 - val_acc: 0.8500
Epoch 8/10
80/80 [===========================

In [53]:
# test data
results=model.evaluate(X_test_norm, Y_test_oneHot)
# `results` holds the results for the model metrics parameters set at the beginning
print(model.metrics_names)
print(results)
print(f"Test results- {model.metrics_names[0]}: {results[0]}, {model.metrics_names[1]}: {results[1]}")

20/20 [==============================] - 0s 400us/step
['loss', 'acc']
[0.15130497515201569, 1.0]
Test results- loss: 0.15130497515201569, acc: 1.0
